Gerekli kütüphaneleri yükleme:

In [1]:

# Gerekli kütüphaneler
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score

from gensim.models import Word2Vec
import warnings
warnings.filterwarnings("ignore")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gultekinqwe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gultekinqwe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Veri setlerini oku
d1 = pd.read_csv("dataFile/spam_modified.csv", sep=';', header=0, usecols=[0, 1])
d2 = pd.read_csv("dataFile/enron_spam_data_modified.csv", sep=';', header=0, usecols=[0, 1])

#sütunlara etiket ve metin şeklinde isimlendirme yapılır
d1.columns = ['label', 'text']
d2.columns = ['label', 'text']
# ham 0, spam 1 şeklinde sütuna dönüşüm yapılır
d1['label'] = d1['label'].str.strip().str.lower().map({'ham': 0, 'spam': 1})
d2['label'] = d2['label'].str.strip().str.lower().map({'ham': 0, 'spam': 1})

#Boş satır ve sütunlar silinir    
d1.dropna(subset=['label', 'text'], inplace=True)
d2.dropna(subset=['label', 'text'], inplace=True)

#Float değerler oluşmuşsa bunları integer a çevirir
d1['label'] = d1['label'].astype(int)
d2['label'] = d2['label'].astype(int)


Dataların ön işleme yapıldığı fonksiyon:

In [3]:
# Ön işleme fonksiyonu 
# Küçük harfe dönüştürme
#Noktalama işaretlerinin kaldırılması
#Stop-word’lerin çıkarılması
#Lemmatization (tercihen)
#Yinelenen e-postaların temizlenmesi
def preprocess(text):
    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()
    
    text = str(text).lower()
    text = re.sub(r"[^a-zA-Z ]", "", text)
    
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)


Özellik çıkarımının kullanıldığı fonksiyon:

In [4]:
# Özellik çıkarımı fonksiyonu
def extract_features(data):
    data["clean_text"] = data["text"].apply(preprocess)
    data = data[data["clean_text"].str.strip() != ""]
    data = data[data["clean_text"].str.len() > 3]

    #TF-IDF vektörlerinin oluştuğu kısım (Her kelimenin sayısal önemini gösteren vektörler üretir)
    tfidf = TfidfVectorizer(max_features=5000)#maximum 5000 kelimeye kadar dikkate alınır
    tfidf_vectors = tfidf.fit_transform(data["clean_text"])

    #LSA kullanıldığı kısım (TF-IDF den gelen matrisin boyutu düşürülür) 100 öznitelik vektörü üretilir
    lsa = TruncatedSVD(n_components=100, random_state=42)
    X_lsa = lsa.fit_transform(tfidf_vectors)
    
    
    # Burda temizlenmiş metinlerden gelen kelimeler benzerliklerine göre benzer vektörlere ayrılır
    sentences = [text.split() for text in data["clean_text"]]
    w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=2)
    word_vectors = w2v_model.wv

    #Word2Vec ile elde edilen kelime vektörleri, 50 kümeye ayrılır. Her küme belirli bir anlam grubunu temsil eder.
    kmeans = KMeans(n_clusters=50, random_state=42)
    kmeans.fit(word_vectors.vectors.astype(np.float64))

    #Her bir metindeki kelimeleri, Word2Vec vektörleri üzerinden KMeans ile oluşturulmuş 50 kümeye atar ve Hangi kümeden kaç kelime geçtiğini sayar.
    #Bu sayım üzerinden 50 boyutlu öznitelik vektörü üretir.
    def get_cluster_features(text):
        cluster_count = np.zeros(50)
        if not isinstance(text, str): return cluster_count
        for word in text.split():
            if word in word_vectors:
                vec = np.asarray(word_vectors[word], dtype=np.float64)
                idx = kmeans.predict([vec])[0]
                cluster_count[idx] += 1
        return cluster_count
    # get_cluster_features fonksiyonu, her metni 50 boyutlu bir vektöre dönüştürür (kelime kümelerine göre)
    semantic_vectors = np.array([get_cluster_features(text) for text in data["clean_text"]])
    # LSA (100 boyut) ve semantic_vectors (50 boyut) birleştirilerek toplam 150 boyutlu öznitelik vektörü elde edilir
    X = np.hstack((X_lsa, semantic_vectors))
    #Etiket vektörü
    y = data["label"].values

    return X, y

Modellerin eğitildiği fonksiyon:

In [5]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, roc_auc_score
from scipy.stats import randint, uniform

######1. DATA SETI ICIN Grid search algoritmasiyla uygun olan parametre secilicek cunku dosya boyutu 2. veri setine gore daha az########
# Eğitim/test ayrımı 
X1, y1 = extract_features(d1)  

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, stratify=y1, random_state=42)

# SVM için Grid Search parametreleri
svm_params = {
    "C": [0.1, 1, 10],
    "kernel": ["linear", "rbf"],
    "gamma": ["scale", "auto"]
}

grid_svm = GridSearchCV(SVC(probability=True), svm_params, cv=2, scoring='f1')
grid_svm.fit(X_train1, y_train1)

# Random Forest için Grid Search parametreleri
rf_params = {
    "n_estimators": [50, 100],
    "max_depth": [None, 10, 20]
}
#RandomForest icin gridSearch yapiliyor
grid_rf = GridSearchCV(RandomForestClassifier(), rf_params, cv=2, scoring='f1')
grid_rf.fit(X_train1, y_train1)

# MLP için Grid Search parametreleri
mlp_params = {
    "hidden_layer_sizes": [(100,), (50, 50)],
    "learning_rate_init": [0.001, 0.01],
    "max_iter": [200]
}
#NLP icin GridSearch yapiliyor
grid_mlp = GridSearchCV(MLPClassifier(), mlp_params, cv=2, scoring='f1')
grid_mlp.fit(X_train1, y_train1)

# Test edilen modeller en iyi parametreleriyle beraber listeye ekleniyor
best_models_D1 = {
    "SVM": SVC(**grid_svm.best_params_, probability=True),
    "Random Forest": RandomForestClassifier(**grid_rf.best_params_),
    "MLP": MLPClassifier(**grid_mlp.best_params_)
}


done


In [6]:

X2, y2 = extract_features(d2)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, stratify=y2, random_state=42)

# Randomized Search Parametreler (optimize edilmiş)
svm_params2 = {
    "C": [0.1, 1],
    "kernel": ["linear"],
    "gamma": ["scale"]
}
rf_params2 = {
    "n_estimators": [50, 100],
    "max_depth": [10]
}
mlp_params2 = {
    "hidden_layer_sizes": [(100,), (50,)],
    "learning_rate_init": [0.001],
    "max_iter": [200]
}

# RandomizedSearch 
random_svm = RandomizedSearchCV(SVC(probability=True), svm_params2, n_iter=5, cv=2, scoring='f1', random_state=42, n_jobs=-1)
random_rf  = RandomizedSearchCV(RandomForestClassifier(), rf_params2, n_iter=5, cv=2, scoring='f1', random_state=42, n_jobs=-1)
random_mlp = RandomizedSearchCV(MLPClassifier(), mlp_params2, n_iter=5, cv=2, scoring='f1', random_state=42, n_jobs=-1)


# 
random_svm.fit(X_train2, y_train2)
random_rf.fit(X_train2, y_train2)
random_mlp.fit(X_train2, y_train2)

# Test edilen modellerin en iyi parametreleriyle beraber listeye ekleniyor
best_models_D2 = {
    "SVM": SVC(**random_svm.best_params_, probability=True),
    "Random Forest": RandomForestClassifier(**random_rf.best_params_),
    "MLP": MLPClassifier(**random_mlp.best_params_)
}    

In [7]:
def results(X_train, y_train, X_test, y_test, models, dataSet_name):
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print(f"\n--- {dataSet_name} | {name} ---")
        print(classification_report(y_test, y_pred))
        print("Accuracy:", accuracy_score(y_test, y_pred))
        print("F1 Score:", f1_score(y_test, y_pred))
        print("ROC AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))
    
    

In [8]:

results(X_train1, y_train1, X_test1, y_test1, best_models_D1, "Spam Modified Dataset")

results(X_train2, y_train2, X_test2, y_test2, best_models_D2, "Enron Dataset")


--- Spam Modified Dataset | SVM ---
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       956
           1       0.96      0.86      0.91       149

    accuracy                           0.98      1105
   macro avg       0.97      0.93      0.95      1105
weighted avg       0.98      0.98      0.98      1105

Accuracy: 0.9764705882352941
F1 Score: 0.9078014184397163
ROC AUC: 0.9701286119457471

--- Spam Modified Dataset | Random Forest ---
              precision    recall  f1-score   support

           0       0.97      1.00      0.99       956
           1       0.99      0.82      0.90       149

    accuracy                           0.97      1105
   macro avg       0.98      0.91      0.94      1105
weighted avg       0.98      0.97      0.97      1105

Accuracy: 0.9746606334841629
F1 Score: 0.8970588235294118
ROC AUC: 0.9660322653112803

--- Spam Modified Dataset | MLP ---
              precision    recall  f1-score   support